# About: CentOS 6 VMイメージ作成

CentOS 6 VMイメージを作成するためのNotebook。

## Operation Note

*ここに経緯を記述*

# Notebookと環境のBinding

Inventory中のgroup名でBind対象を指示する。

**VMを起動したいホスト(KVMなどがインストールされた物理マシン)**を示すInventory中の名前を以下に指定する。

In [1]:
target_group = 'test-hypervisor'

Bind対象への疎通状態を確認する。

In [2]:
!ansible -m ping {target_group}

XXX.XXX.XXX.105 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


Bind対象は以下の条件を満たしている必要がある。**満たしていない場合は、このお手本の操作をBind対象にそのまま適用することはできず、適宜セルの改変が必要。**

## 仮想マシン用ブリッジが作成されていること

仮想マシン用のブリッジが作成されていること。お手本を作成している環境においては、以下のようなインタフェース構成となることを想定している。

- ブリッジ br-eth1 インタフェース ... ここにはサービス用IPアドレスが設定される
- eth1インタフェース ... Promiscuousモードでサービス用NICと対応付け、br-eth1インタフェースに接続される

In [3]:
external_nic = 'eth1'
bridge_nic = 'br-eth1'

In [4]:
!ansible -a "/sbin/ip addr show {bridge_nic}" {target_group}
!ansible -a "/sbin/ip addr show {external_nic}" {target_group}
!ansible -a "/usr/sbin/brctl show {bridge_nic}" {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
10: br-eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UNKNOWN 
    link/ether XX:XX:XX:XX:XX:XX brd XX:XX:XX:XX:XX:XX
    inet XXX.XXX.XXX.105/26 brd XXX.XXX.XXX.127 scope global br-eth1
    inet6 XX:XX:XX:XX:XX:XX/64 scope link 
       valid_lft forever preferred_lft forever

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
2: eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP qlen 1000
    link/ether XX:XX:XX:XX:XX:XX brd XX:XX:XX:XX:XX:XX
    inet6 XX:XX:XX:XX:XX:XX/64 scope link 
       valid_lft forever preferred_lft forever

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
bridge name	bridge id		STP enabled	interfaces
br-eth1		8000.246e960db538	no		eth1



ブリッジ用NIC名として br-eth1 を利用する。

**br-eth1, eth1が定義されており、br-eth1にサービス用IPアドレスが定義されていれば**OK。

## libvirtのNetwork設定が無効化されていること

defaultのNetwork設定が無効化されているかどうかを確認する。

In [5]:
!ansible -b -a 'virsh net-list --all' {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
Name                 State      Autostart     Persistent
--------------------------------------------------
default              inactive   no            yes



**defaultのstateがinactiveになっていて、かつautostartがnoになっていれば**OK。

## dnsmasqが起動していること

同じホストで、IPアドレス配布用のdnsmasqが実行されていることを前提としている。

In [6]:
!ansible -b -a 'service dnsmasq status' {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
dnsmasq (pid  9388) is running...dnsdomainname: Unknown host



**dnsmasq (pid  XXXXX) is running と表示されれば**OK。

## libvirtが動作していること

libvirtが動作しており、仮想マシン一覧が取得できるかどうかを確認する。

In [7]:
!ansible -b -a 'virsh list' {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
 Id    Name                           State
----------------------------------------------------



**エラーメッセージが表示されなければ**OK。

## virt-installがインストールされていること

仮想マシンの作成には、virt-installコマンドを利用する。

In [8]:
!ansible -a 'which virt-install' {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
/usr/bin/virt-install



**エラーメッセージが表示されなければ**OK。

# パラメータの決定

イメージ作成により、以下の2つのファイルがBinding対象ホストに作成される。

- base.img
- libvirt-base.xml

このファイルを作成するディレクトリのパスと、イメージのサイズ(GB)を指定する。

In [9]:
image_base_dir = '/mnt/centos6-base-vm'
size_gb = 100

`size_gb` で指定した空き容量がBind対象ホストにあるかどうかを確認する。

In [10]:
!ansible -a 'df -H' {target_group} 

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
Filesystem      Size  Used Avail Use% Mounted on
/dev/sda2        14G  2.8G   11G  21% /
tmpfs            68G     0   68G   0% /dev/shm
/dev/sda5       1.7T  104M  1.7T   1% /mnt



# イメージ取得用VMの新規作成

Binding対象ホストにイメージ保存用のディレクトリを作成する。

In [11]:
!ansible -b -m file -a 'path={image_base_dir} state=directory' {target_group} 

XXX.XXX.XXX.105 | SUCCESS => {
    "changed": true, 
    "gid": 0, 
    "group": "root", 
    "mode": "0755", 
    "owner": "root", 
    "path": "/mnt/centos6-base-vm", 
    "size": 4096, 
    "state": "directory", 
    "uid": 0
}


スナップショット用のVM名を決める。

In [12]:
new_vmname = 'snapshot-vm-20160609'

仮想マシンの作成は、virt-installを使い、CentOS 6のMinimal ISOを使ってインストールする。

## インストール用ISOの準備

インストールディスクのダウンロードをおこなう。

MD5チェックサムが `0ca12fe5f28c2ceed4f4084b41ff8a0b` であることを確認すること。*(2016/06/17)*

In [13]:
!ansible -b -m get_url -a 'url=http://ftp.riken.jp/Linux/centos/6/isos/x86_64/CentOS-6.8-x86_64-minimal.iso \
                           dest=/tmp/CentOS-6.8-x86_64-minimal.iso' {target_group}

XXX.XXX.XXX.105 | SUCCESS => {
    "changed": true, 
    "checksum_dest": null, 
    "checksum_src": "28cd663c2267676414496f0929ce7bb285bf2506", 
    "dest": "/tmp/CentOS-6.8-x86_64-minimal.iso", 
    "gid": 0, 
    "group": "root", 
    "md5sum": "0ca12fe5f28c2ceed4f4084b41ff8a0b", 
    "mode": "0644", 
    "msg": "OK (468713472 bytes)", 
    "owner": "root", 
    "size": 468713472, 
    "src": "/tmp/tmpKQONEl", 
    "state": "file", 
    "uid": 0, 
    "url": "http://ftp.riken.jp/Linux/centos/6/isos/x86_64/CentOS-6.8-x86_64-minimal.iso"
}


## Kickstartファイルの準備

インストール手順は Kickstartを使って定義する。

念のため、VMにはrootパスワードを指定しておく。このパスワードはスナップショット処理の最後にロックする。

In [14]:
from getpass import getpass
rootpw = getpass()

········


CentOS6のインストールをおこない、public keyをInjectionするようなKickstartファイルを生成する。

まずローカルに一時ディレクトリを作り、そこにファイルを作成する。

In [15]:
import tempfile
work_dir = tempfile.mkdtemp()
work_dir

'/tmp/tmp5XssOP'

In [16]:
import os
pub_key = None
with open(os.path.expanduser('~/.ssh/ansible_id_rsa.pub'), 'r') as f:
    pub_key = f.readlines()[0].strip()

with open(os.path.join(work_dir, 'centos6.ks.cfg'), 'w') as f:
    f.write('''#version=RHEL6

cmdline
cdrom
install

lang en_US.UTF-8
keyboard jp106

network --device eth0 --onboot yes --bootproto dhcp --noipv6

zerombr
bootloader --location=mbr --append="crashkernel=auto rhgb quiet"

clearpart --all --initlabel
part / --fstype=ext4 --grow --size=1 --asprimary

rootpw --plaintext {rootpw}
authconfig --enableshadow --passalgo=sha512
selinux --disabled
firewall --disabled
firstboot --disabled
timezone --utc Asia/Tokyo

poweroff

%packages --nobase
%end

%post
cd /root
mkdir --mode=700 .ssh
cat >> .ssh/authorized_keys << "PUBLIC_KEY"
{pub_key}
PUBLIC_KEY
chmod 600 .ssh/authorized_keys
%end'''.format(rootpw=rootpw, pub_key=pub_key))

!grep -v rootpw {work_dir}/centos6.ks.cfg

#version=RHEL6

cmdline
cdrom
install

lang en_US.UTF-8
keyboard jp106

network --device eth0 --onboot yes --bootproto dhcp --noipv6

zerombr
bootloader --location=mbr --append="crashkernel=auto rhgb quiet"

clearpart --all --initlabel
part / --fstype=ext4 --grow --size=1 --asprimary

authconfig --enableshadow --passalgo=sha512
selinux --disabled
firewall --disabled
firstboot --disabled
timezone --utc Asia/Tokyo

poweroff

%packages --nobase
%end

%post
cd /root
mkdir --mode=700 .ssh
cat >> .ssh/authorized_keys << "PUBLIC_KEY"
ssh-rsa XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX ansible@XXXXXXXXXXXX
PUBLIC_KEY
chmod 600 .ssh/authorized_keys
%end


なお、Kickstartの設定では、最後にpoweroffすることでインストール成功後、VMを停止するようにしている。

Bind対象にアップロードする。

In [17]:
!ansible -b -m copy -a 'src={work_dir}/centos6.ks.cfg dest=/tmp/centos6.ks.cfg' {target_group}

XXX.XXX.XXX.105 | SUCCESS => {
    "changed": true, 
    "checksum": "e7ca71f3e57085dcbb8df4a3b909c4ae52c7e96f", 
    "dest": "/tmp/centos6.ks.cfg", 
    "gid": 0, 
    "group": "root", 
    "md5sum": "c4e9f7b255cc3f9b2fcf68566bee7b99", 
    "mode": "0644", 
    "owner": "root", 
    "size": 1017, 
    "src": "/home/ansible/.ansible/tmp/ansible-tmp-1466157873.72-86285770392274/source", 
    "state": "file", 
    "uid": 0
}


## インストールの実行

virt-installを実行する。なお、AnsibleのSSH処理の関係で、 `process.error: Cannot run interactive console without a controlling TTY` と出力されるが、ここでは無視する。

In [18]:
!ansible -b -a 'virt-install --name {new_vmname} \
                             --hvm \
                             --virt-type kvm \
                             --ram 1024 \
                             --vcpus 1 \
                             --arch x86_64 \
                             --os-type linux \
                             --os-variant rhel6 \
                             --boot hd \
                             --disk path\={image_base_dir}/base.img,size\={size_gb},format\=raw \
                             --network bridge\={bridge_nic} \
                             --graphics none \
                             --serial pty \
                             --console pty \
                             --noreboot \
                             --location /tmp/CentOS-6.8-x86_64-minimal.iso \
                             --initrd-inject /tmp/centos6.ks.cfg \
                             --extra-args "ks\=file:/centos6.ks.cfg console\=ttyS0"' {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>

Starting install...
Retrieving file .treeinfo...                             |  676 B     00:00 ... 
Retrieving file vmlinuz...                               | 8.1 MB     00:00 ... 
Retrieving file initrd.img...                            |  78 MB     00:00 ... 
Creating storage file base.img                           | 100 GB     00:00     
Creating domain...                                       |    0 B     00:00     

Domain installation still in progress. You can reconnect to 
the console to complete the installation process.error: Cannot run interactive console without a controlling TTY



VMの状態確認は以下で行える。

In [19]:
!ansible -b -m shell -a 'virsh dominfo {new_vmname} | grep State' {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
State:          running



具体的なコンソール出力の確認は、 `virsh console ${new_vmname}` でもおこなえる。

poweroffされるまで待つ・・・

In [20]:
vm_status = !ansible -b -m shell -a 'virsh dominfo {new_vmname} | grep State' {target_group}

import time
while vm_status[1].split()[-1] == 'running':
    time.sleep(60)
    vm_status = !ansible -b -m shell -a 'virsh dominfo {new_vmname} | grep State' {target_group}

以下の出力が `shut off` となっていればOK。

In [21]:
!ansible -b -m shell -a 'virsh dominfo {new_vmname} | grep State' {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
State:          shut off



起動してみる。

In [22]:
!ansible -b -a 'virsh start {new_vmname}' {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
Domain snapshot-vm-20160609 started



## 仮想マシンの情報確認

VMにふられたIPアドレスの確認

In [23]:
!ansible -b -a "virsh domiflist {new_vmname}" {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
Interface  Type       Source     Model       MAC
-------------------------------------------------------
vnet0      bridge     br-eth1    virtio      XX:XX:XX:XX:XX:XX



上記で確認できたMACアドレスを、以下の変数に代入。

In [24]:
import re
domiflist_stdio = !ansible -b -a "virsh domiflist {new_vmname}" {target_group}
mac_pattern = re.compile(r'.*bridge.*\s([0-9a-f\:]+)\s*')
vmmac = [mac_pattern.match(line).group(1) for line in domiflist_stdio if mac_pattern.match(line)][0]
vmmac

'XX:XX:XX:XX:XX:XX'

dnsmasqのlease情報を確認する。

In [25]:
!ansible -b -a "grep {vmmac} /var/lib/dnsmasq/dnsmasq.leases" {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
1466201126 XX:XX:XX:XX:XX:XX XXX.XXX.XXX.66 * *



In [26]:
leases_stdio = !ansible -b -a "grep {vmmac} /var/lib/dnsmasq/dnsmasq.leases" {target_group}
ip_pattern = re.compile(r'.*\s([0-9a-f\:]+)\s+([0-9\.]+)\s.*')
ipaddr = [ip_pattern.match(line).group(2) for line in leases_stdio if ip_pattern.match(line)][0]
ipaddr

'XXX.XXX.XXX.66'

このIPアドレスに対して操作すればよい・・・疎通しているか、確認する。

(VMには、このNotebook環境から疎通するIPアドレスが振られることを想定している。)

In [27]:
!ping -c 4 {ipaddr}

PING XXX.XXX.XXX.66 (XXX.XXX.XXX.66) 56(84) bytes of data.
64 bytes from XXX.XXX.XXX.66: icmp_seq=1 ttl=63 time=6.80 ms
64 bytes from XXX.XXX.XXX.66: icmp_seq=2 ttl=63 time=0.242 ms
64 bytes from XXX.XXX.XXX.66: icmp_seq=3 ttl=63 time=0.345 ms
64 bytes from XXX.XXX.XXX.66: icmp_seq=4 ttl=63 time=0.232 ms

--- XXX.XXX.XXX.66 ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 2999ms
rtt min/avg/max/mdev = 0.232/1.905/6.803/2.828 ms


## 仮想マシンの設定変更



### Ansible操作用ユーザの作成

ユーザ `ansible` でAnsibleの操作が可能なよう、設定変更をおこなう。

In [28]:
import os
snapshot_hosts = os.path.join(work_dir, 'init-hosts')
with open(snapshot_hosts, 'w') as f:
    f.write('{address} ansible_ssh_user=root\n'.format(address=ipaddr)) 
!cat { snapshot_hosts }

XXX.XXX.XXX.66 ansible_ssh_user=root


Ansible経由でpingできるかの確認をする。

In [29]:
!ansible -m ping -i { snapshot_hosts } all

XXX.XXX.XXX.66 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


設定変更用のPlaybookを生成する。

In [30]:
pub_key = None
with open(os.path.expanduser('~/.ssh/ansible_id_rsa.pub'), 'r') as f:
    pub_key = f.readlines()[0].strip()

playbook_inject_key = os.path.join(work_dir, 'playbook_inject-key.yml')
with open(playbook_inject_key, 'w') as f:
    f.write('- hosts: all\n')
    f.write('  become: yes\n')
    f.write('  tasks:\n')
    f.write('    - user: name=ansible state=present\n')
    f.write('    - authorized_key: user=ansible key="{}"\n'.format(pub_key))
    f.write('    - lineinfile: "dest=/etc/sudoers backup=yes state=present regexp=\'^ansible\' line=\'ansible ALL=(ALL) NOPASSWD: ALL\'"\n')
    f.write('    - command: passwd -l root\n')
    
!cat { playbook_inject_key }

- hosts: all
  become: yes
  tasks:
    - user: name=ansible state=present
    - authorized_key: user=ansible key="ssh-rsa XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX ansible@XXXXXXXXXXXX"
    - lineinfile: "dest=/etc/sudoers backup=yes state=present regexp='^ansible' line='ansible ALL=(ALL) NOPASSWD: ALL'"
    - command: passwd -l root


Playbookを実行する。

In [31]:
!ansible-playbook -i { snapshot_hosts } { playbook_inject_key }


PLAY [all] *********************************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.66]

TASK [user] ********************************************************************
changed: [XXX.XXX.XXX.66]

TASK [authorized_key] **********************************************************
changed: [XXX.XXX.XXX.66]

TASK [lineinfile] **************************************************************
changed: [XXX.XXX.XXX.66]

TASK [command] *****************************************************************
changed: [XXX.XXX.XXX.66]

PLAY RECAP *********************************************************************
XXX.XXX.XXX.66               : ok=5    changed=4    unreachable=0    failed=0   



これで、ユーザ `ansible` でSSH可能な状態になった。

In [32]:
snapshot_hosts = os.path.join(work_dir, 'hosts')
with open(snapshot_hosts, 'w') as f:
    f.write('{address}\n'.format(address=ipaddr)) 
!cat { snapshot_hosts }

XXX.XXX.XXX.66


以下のpingに成功すればOK。

In [33]:
!ansible -m ping -i { snapshot_hosts } all

XXX.XXX.XXX.66 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


### ifcfgの修正

インタフェースの定義にスナップショット時のMACアドレスが含まれていると、新規VMとして起動したときに正しくNIC設定が適用されないので、修正しておく。

In [34]:
!ansible -a 'cat /etc/sysconfig/network-scripts/ifcfg-eth0' -i { snapshot_hosts } all

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
DEVICE="eth0"
BOOTPROTO="dhcp"
HWADDR="XX:XX:XX:XX:XX:XX"
IPV6INIT="no"
NM_CONTROLLED="yes"
ONBOOT="yes"
TYPE="Ethernet"
UUID="9fdb4435-95fd-4a29-b4a9-67ab00ae4d51"



`HWADDR`, `UUID`の定義を削除する。

In [35]:
!ansible -b -m lineinfile -a "dest=/etc/sysconfig/network-scripts/ifcfg-eth0 regexp='^HWADDR=' state=absent" -i { snapshot_hosts } all
!ansible -b -m lineinfile -a "dest=/etc/sysconfig/network-scripts/ifcfg-eth0 regexp='^UUID=' state=absent" -i { snapshot_hosts } all

XXX.XXX.XXX.66 | SUCCESS => {
    "backup": "", 
    "changed": true, 
    "found": 1, 
    "msg": "1 line(s) removed"
}
XXX.XXX.XXX.66 | SUCCESS => {
    "backup": "", 
    "changed": true, 
    "found": 1, 
    "msg": "1 line(s) removed"
}


`HWADDR`, `UUID`の定義が削除されていればよい。

In [36]:
!ansible -a 'cat /etc/sysconfig/network-scripts/ifcfg-eth0' -i { snapshot_hosts } all

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
DEVICE="eth0"
BOOTPROTO="dhcp"
IPV6INIT="no"
NM_CONTROLLED="yes"
ONBOOT="yes"
TYPE="Ethernet"



### udevのネットワーク定義の修正

udevの定義も削除しておく。

In [37]:
!ansible -a 'cat /etc/udev/rules.d/70-persistent-net.rules' -i { snapshot_hosts } all

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
# This file was automatically generated by the /lib/udev/write_net_rules
# program, run by the persistent-net-generator.rules rules file.
#
# You can modify it, as long as you keep each rule on a single
# line, and change only the value of the NAME= key.

# PCI device 0x1af4:0x1000 (virtio-pci)
SUBSYSTEM=="net", ACTION=="add", DRIVERS=="?*", ATTR{address}=="XX:XX:XX:XX:XX:XX", ATTR{type}=="1", KERNEL=="eth*", NAME="eth0"



In [38]:
!ansible -b -m file -a 'path=/etc/udev/rules.d/70-persistent-net.rules state=absent' -i { snapshot_hosts } all

XXX.XXX.XXX.66 | SUCCESS => {
    "changed": true, 
    "path": "/etc/udev/rules.d/70-persistent-net.rules", 
    "state": "absent"
}


削除されているかを確認する。

In [39]:
!ansible -a 'ls -la /etc/udev/rules.d/' -i { snapshot_hosts } all

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
total 20
drwxr-xr-x. 2 root root 4096 Jun 17 19:09 .
drwxr-xr-x. 4 root root 4096 Jun 17 19:05 ..
-rw-r--r--. 1 root root  316 May 11 07:58 60-raw.rules
-rw-r--r--  1 root root  536 Jun 17 19:08 70-persistent-cd.rules
-rw-r--r--. 1 root root   54 May 11 16:38 99-fuse.rules



## VMイメージファイルへの同期

In [40]:
!ansible -a 'sync' -i { snapshot_hosts } all

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>




# VM定義の保存

VM複製用に、XML定義を得ておく。

In [41]:
import xml.etree.ElementTree as ET
vmxml_s = !ansible -b -a "virsh dumpxml {new_vmname}" {target_group}
vmxml_s = vmxml_s[1:]
vmxml = ET.fromstring('\n'.join(vmxml_s))

del vmxml.attrib['id']
vmxml.remove(vmxml.find('uuid'))
intrElem = vmxml.find('devices').find('interface')
intrElem.remove(intrElem.find('target'))
intrElem.remove(intrElem.find('alias'))

vmxml.find('name').text = ''
vmxml.find('devices').find('disk').find('source').attrib['file'] = ''
vmxml.find('devices').find('interface').find('mac').attrib['address'] = ''

ET.ElementTree(vmxml).write(os.path.join(work_dir, 'libvirt-base.xml'))
!cat {work_dir}/libvirt-base.xml

<domain type="kvm">
  <name />
  <memory unit="KiB">1048576</memory>
  <currentMemory unit="KiB">1048576</currentMemory>
  <vcpu placement="static">1</vcpu>
  <os>
    <type arch="x86_64" machine="rhel6.6.0">hvm</type>
    <boot dev="hd" />
  </os>
  <features>
    <acpi />
    <apic />
    <pae />
  </features>
  <clock offset="utc" />
  <on_poweroff>destroy</on_poweroff>
  <on_reboot>restart</on_reboot>
  <on_crash>restart</on_crash>
  <devices>
    <emulator>/usr/libexec/qemu-kvm</emulator>
    <disk device="disk" type="file">
      <driver cache="none" name="qemu" type="raw" />
      <source file="" />
      <target bus="virtio" dev="vda" />
      <alias name="virtio-disk0" />
      <address bus="0x00" domain="0x0000" function="0x0" slot="0x05" type="pci" />
    </disk>
    <disk device="cdrom" type="block">
      <driver name="qemu" type="raw" />
      <target bus="ide" dev="hdc" />
      <readonly />
      <alias name="ide0-1-0" />
      <address bus="1" controller="0" target="0"

リモートのイメージと同じパスに保存しておく。

In [42]:
!ansible -b -m copy -a 'src={work_dir}/libvirt-base.xml dest={image_base_dir}' {target_group}

XXX.XXX.XXX.105 | SUCCESS => {
    "changed": true, 
    "checksum": "d42215623ad5fca3424317f7d2623fb48b09fdd7", 
    "dest": "/mnt/centos6-base-vm/libvirt-base.xml", 
    "gid": 0, 
    "group": "root", 
    "md5sum": "f34e277e56387fadb8144b5b67139949", 
    "mode": "0644", 
    "owner": "root", 
    "size": 2839, 
    "src": "/home/ansible/.ansible/tmp/ansible-tmp-1466158206.07-173445812115609/source", 
    "state": "file", 
    "uid": 0
}


# イメージ取得用VMの停止

停止してBaseの作業完了・・・

In [43]:
!ansible -b -a "virsh destroy {new_vmname}" {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
Domain snapshot-vm-20160609 destroyed



しばらく待ってから再度 virsh listを実行すると、仮想マシンが停止してリストから消えたことがわかる。

In [44]:
!ansible -b -a "virsh list" {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
 Id    Name                           State
----------------------------------------------------



VMの定義も削除しておく。

In [45]:
!ansible -b -a "virsh undefine {new_vmname}" {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
Domain snapshot-vm-20160609 has been undefined



## dnsmasqの後始末

dnsmasqのリース情報の後始末。VM用IPアドレスが潤沢にある場合は不要。

In [46]:
!ansible -a "cat /var/lib/dnsmasq/dnsmasq.leases" {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
1466201307 XX:XX:XX:XX:XX:XX XXX.XXX.XXX.66 localhost *



In [47]:
!ansible -b -m lineinfile -a "dest=/var/lib/dnsmasq/dnsmasq.leases regexp='^.*\s+{ ipaddr }\s+.*' state=absent" {target_group}

XXX.XXX.XXX.105 | SUCCESS => {
    "backup": "", 
    "changed": true, 
    "found": 1, 
    "msg": "1 line(s) removed"
}


In [48]:
!ansible -a "cat /var/lib/dnsmasq/dnsmasq.leases" {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>




In [49]:
!ansible -b -m service -a "name=dnsmasq state=restarted" {target_group}

XXX.XXX.XXX.105 | SUCCESS => {
    "changed": true, 
    "name": "dnsmasq", 
    "state": "started"
}


# イメージファイルの確認

イメージファイルとXML定義が生成されていることを確認する。以下の2つのファイルがホストに作成されていればOK。

- base.img
- libvirt-base.xml

In [50]:
!ansible -b -a "ls -la {image_base_dir}" {target_group}

XXX.XXX.XXX.105 | SUCCESS | rc=0 >>
total 2691380
drwxr-xr-x 2 root root         4096 Jun 17 19:10 .
drwxr-xr-x 5 root root         4096 Jun 17 19:02 ..
-rwxr-xr-x 1 root root 107374182400 Jun 17 19:10 base.img
-rw-r--r-- 1 root root         2839 Jun 17 19:10 libvirt-base.xml



完了。

# 後始末

一時ディレクトリを削除する。

In [51]:
!rm -fr {work_dir}